# Memory Layout

This notebook deals with the relationship between performance and how effectively you exploit the computer's memory hierarchy. 

There are three major points you need to remember: 

1. When CPU runs an instruction it operates on things in the registers. If something is not in the CPU registers, the CPU needs to fetch the data from memory: this is SLOW!!!
2. The CPU first looks at L1 cache, then L2 cache then Main memory and then Swap space - L1, L2 caches are still small (order of megabytes), but hitting them often will give orders of magnitude performance gain as compared to hitting main memory that often
3. The computer optimistically brings things from main memory onto the caches when you access a chunk of data. Hence if you access data that is continuous in memory, they all get asynchronously brought into the cache and your program will be really fast.

### A brief aside: `mutable struct`s

So far, we've dealt only with `struct`s. The problem with structs is that they don't let you change its contents. Consider the following example:

In [ ]:
struct Thing
    a::Int
end

In [ ]:
x = Thing(1)
x.a = 2

If you'd like to change the value stored in your object, you must `mutable struct`s instead. 

In [ ]:
mutable struct Thing2
    a::Int
end

In [ ]:
x = Thing2(1)
x.a = 2
x

This looks convenient! Let's continue exploring `mutable struct`s for a bit.

In [ ]:
abstract type TestType end

`abstract` types are types you can use represent abstract concepts. 

For example, "car" can be an abstract type, which doesn't commit to any specifics, while concrete types like "Porche" would be very specific.

Now, types can have subtypes. Recall `Integer` represents a bunch a different Integer types. More specifically, `Int64` and `Int32` are **subtypes** of `Int64` and conversely, `Integer` is the **supertype** of `Int64` and `Int32`. 

Okay, now let's create a subtype of `TestType`: 

In [ ]:
mutable struct Typ <: TestType
    x::Int16
    y::Int16
end

We already know how to overload `+`.

In [ ]:
Base.:+(a::T, b::T) where {T<:TestType} = T(a.x+b.x, a.y+b.y)

In [ ]:
Typ(2,2) + Typ(4,4)

Now let's start thinking about sizes, specifically: sizes of objects in memory. 

The size of `Typ` is 4 bytes.

In [ ]:
sizeof(Typ(2,2))

In [ ]:
@time typ_arr = [Typ(i%127,i%127) for i=1:10^6];

Notice the allocation. 


In [ ]:
sizeof(typ_arr)

**Claim**: This array is bigger than it should be!! Can you see why?

In [ ]:
sizeof(typ_arr) / 10^6 # bytes per object

This is because mutable objects are **passed by reference**!! The objects are being "boxed".

**Discuss**: why are they being passed by reference?

Sum could also have been much more efficient....

In [ ]:
@time sum(typ_arr)

In [ ]:
struct Imm <: TestType
    x::Int16
    y::Int16
end

In [ ]:
sizeof(Imm(2,2))

In [ ]:
@time imm_arr = [Imm(i%127,i%127) for i=1:10^6];

In [ ]:
sizeof(imm_arr)

**Seems correct!**

Since structs can never be changed, their value _is_ their identity, the compiler can **pass them by value**

In [ ]:
Base.:+(a::Imm, b::Imm) = Imm(a.x+b.x, a.y+b.y)

In [ ]:
@time sum(imm_arr)

In [ ]:
x = rand(10^6)

@time sum(x)

The compiler can do this optimization because it knows someone else won't be changing the insides of the `Imm` object!

If you don't know the type of the insides of an immutable type, you can tack on a type parameter.

For example

In [ ]:
struct ImmParam{T} <: TestType
    x::T
    y::T
end

In [ ]:
sizeof(ImmParam{Int128}) # sizeof also works on the 

In [ ]:
sizeof(ImmParam{Int8})

In [ ]:
ImmParam{Int8} == ImmParam{Int64}

In [ ]:
ImmParam(1.0,2.0) # Julia can automatically infer this

In [ ]:
ImmParam(1,2)

In [ ]:
ImmParam(1.0,2) # Type parameter ensures that both remain same type

**Memory alignment is tight!** This means this entire array is store continguously in memory. 

In [ ]:
@time imm_par_array_int16 = [ImmParam{Int16}(2,3) for i = 1:10^6];

In [ ]:
sizeof(imm_par_array_int16)

In [ ]:
@time imm_par_array_int8 = [ImmParam{Int8}(2,3) for i = 1:10^6];

In [ ]:
sizeof(imm_par_array_int8)

In [ ]:
using Interact

In [ ]:
mutable struct TypParam{T} <: TestType
    x::T
    y::T
end

In [ ]:
@manipulate for param = [Int8,Int16,Int32,Int64,Float16,Float32,Float64], complex=true
    T = complex ? Complex{param} : param
    a = zero(T)
    b = one(T)

    GC.gc()  
    local arr,t_create,arr_t,t_create_t,t_sum,t_sum_t

    alloc_create = @allocated begin
        t_create = @elapsed begin
            arr = [ImmParam(a,b) for i=1:10^6]
        end
    end

    alloc_create_t = @allocated begin
        t_create_t = @elapsed begin
            arr_t = [TypParam(a,b) for i=1:10^6]
        end
    end

    GC.gc()
    
    alloc_sum = @allocated begin
        t_sum = @elapsed begin
            s = sum(arr)
        end
    end/10^6

    alloc_sum_t = @allocated begin
        t_sum_t = @elapsed begin
            s_t = sum(arr_t)
        end
    end/10^6
    HTML("""<table width="100%" cellspacing="2" cellpadding="0" border="0" align="center" bgcolor="#ff6600" ><tbody>
        <thead>
        <tr>
            <th colspan=4>struct</th> 
            <th colspan=4>mutable struct</th>
        </tr>
        <tr>
            <th colspan=2>create</th> 
            <th colspan=2>sum</th>
            <th colspan=2>create</th> 
            <th colspan=2>sum</th>
        </tr>
        <tr>
            <th>time</th>
            <th>memory</th>
            <th>time</th>
            <th>memory</th>
            <th>time</th>
            <th>memory</th>
            <th>time</th>
            <th>memory</th>

        </tr>
        </thead>
        <tr>
            <td>$t_create</td>
            <td>$alloc_create</td>
            <td>$t_sum</td>
            <td>$alloc_sum</td>
            <td>$t_create_t</td>
            <td>$alloc_create_t</td>
            <td>$t_sum_t</td>
            <td>$alloc_sum_t</td>
        </tr>
        </tbody></table>""")
end

## But be careful! Vectors of Heterogeneous types force boxing!


In [ ]:
["xyzabc", 1+2im, 1, 1.0]

In [ ]:
[ImmParam(UInt8(1),UInt8(1)), ImmParam(1.0,1.0)] |> sizeof

In [ ]:
@time heter_arr = [i%2 == 0 ? ImmParam(UInt8(1),UInt8(1)) : ImmParam(1.0,1.0) for i = 1:10^6]

In [ ]:
@time sum(heter_arr) # why doesn't this work?

## Summary

- Use immutables wherever you consider something to be a *value*. Use type when something is a *state*.
- Never create a large array of mutable objects! Each one is heap-allocated, this kills performance and gives the GC a hard time.
- Parameterize if you need to change types
- In the wizard's own words: http://julialang.org/blog/2013/03/efficient-aggregates